In [2]:
# raw_data = np.array(df['text'])
# labels = [1]*(training_size//2) + [0]*(training_size//2)

In [3]:
training_size = 1600000
data_folder = 'trainingandtestdata'
train_file = './'+data_folder+'/'+'training.csv'
test_file = './' + data_folder+'/'+'testing.csv'

In [4]:
from nltk.stem import LancasterStemmer
# from nltk.tokenize import word_tokenize
# from nltk import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk import bigrams
tk = RegexpTokenizer('^(@)', gaps = True)

import pandas as pd
import numpy as np
lancaster=LancasterStemmer()

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('be')

In [5]:
df = pd.read_csv(train_file)
del df['Id']
del df['Query']
del df['Username']
del df['Date']

def preprocess(df):
    df['text'] = df['text'].apply(lambda x: x.lower())
    df['text'] = df['text'].apply(lambda x: x.replace('.',' '))
    df['text'] = df['text'].apply(lambda x: x.replace('\n',' '))
    df['text'] = df['text'].apply(lambda x: x.replace('?',' '))
    df['text'] = df['text'].apply(lambda x: x.replace('!',' '))
    df['text'] = df['text'].apply(lambda x: x.replace('"',' '))
    df['text'] = df['text'].apply(lambda x: x.replace(';',' '))
    df['text'] = df['text'].apply(lambda x: x.replace('#',''))
    df['text'] = df['text'].apply(lambda x: x.replace(',',' '))
    df['text'] = df['text'].apply(lambda x: x.replace('\'',' '))
    
preprocess(df)

positive = df[df['Y']==4]
negative = df[df['Y']==0]

In [ ]:
array = positive['text'].str.split(' ', expand=True).stack().value_counts()
d = {'word': array.index, 'frequency':array}
# df2 = pd.DataFrame(data = d)

In [ ]:
pos_voc = pd.DataFrame(data = d)

In [ ]:
array = negative['text'].str.split(' ', expand=True).stack().value_counts()
d = {'word': array.index, 'frequency':array}
neg_voc = pd.DataFrame(data = d)

In [ ]:
total_voc_size = pos_voc['frequency'][0] + neg_voc['frequency'][0]
pos_voc['word'] = pos_voc['word'][1:]
pos_voc['frequency'] = pos_voc['frequency'][1:]

neg_voc['word'] = neg_voc['word'][1:]
neg_voc['frequency'] = neg_voc['frequency'][1:]

In [ ]:
total_voc_size

In [ ]:
vocab = {'word': [], 'frequency': []}
for i in range(1, pos_voc['word'].size):
    if(pos_voc['word'][i] in vocab['word']):
        index =  vocab['word'].index(pos_voc['word'][i])
        vocab['frequency'][index] += pos_voc['frequency'][i]
    else:
        vocab['word'].append(pos_voc['word'][i])
        vocab['frequency'].append(pos_voc['frequency'][i])
    print(i)
print('HALF DONE')

for i in range(1, neg_voc['word'].size):
    if(neg_voc['word'][i] in vocab['word']):
        index =  vocab['word'].index(neg_voc['word'][i])
        vocab['frequency'][index] += neg_voc['frequency'][i]
    else:
        vocab['word'].append(neg_voc['word'][i])
        vocab['frequency'].append(neg_voc['frequency'][i])
df1 = pd.DataFrame(vocab)

In [52]:
def log_likely(thetas_given_negative, thetas_given_positive, phis, string):
    lst = string.split(' ')

    log_y_0 = math.log(phis[4])
    for i in range(len(lst)):
        lst[i] = lst[i].lower()
        if(lst[i] in thetas_given_positive):
            log_y_0 += math.log(thetas_given_positive[lst[i]])


    log_y_1 = math.log(phis[0])
    for i in range(len(lst)):
        lst[i] = lst[i].lower()
        if(lst[i] in thetas_given_negative):
            log_y_1 += math.log(thetas_given_negative[lst[i]])
    
    ss = log_y_0 + log_y_1
    log_y_0 /= ss
    log_y_1 /= ss
    if(log_y_0>log_y_1):
        return 0
    return 4
def calculate_phi(df0):
    phis = {}
    df_positive = df0[df0['Y']==4]
    df_negative = df0[df0['Y']==0]

    phis[0] = df_negative.size/6
    phis[4] = df_positive.size/6
    
    return phis
def calculate_theta_given_y(df1, df2, df3):
    thetas_given_positive = {}
    thetas_given_negative = {}

    for i in range(1,df2['frequency'].size):
        thetas_given_positive[df2['word'][i]] = (1+df2['frequency'][i])/(df1['frequency'].size + df2['frequency'][1:].sum())

    for i in range(1,df3['frequency'].size):
        thetas_given_negative[df3['word'][i]] = (1+df3['frequency'][i])/(df1['frequency'].size + df3['frequency'][1:].sum())
    return thetas_given_positive, thetas_given_negative
def train(df0, df1, df2, df3):
    phis = calculate_phi(df0)
    thetas_given_positive, thetas_given_negative = calculate_theta_given_y(df1, df2, df3)
    return phis, thetas_given_positive, thetas_given_negative
def test(phis, thetas_given_positive, thetas_given_negative, test_file):
    test_result = []
    df_test = pd.read_csv(test_file)
    for i in range(df_test['Y'].size):
        result = log_likely(thetas_given_negative, thetas_given_positive, phis, df_test['text'][i])
        test_result.append(result)
    df_test['Y_test'] = test_result
    
    accuracy = 0
    total = 0
    confusion_martix = [[0,0],[0,0]]
    for i in range(df_test['Y'].size):
        if(df_test['Y'][i]!=2):
            if(df_test['Y'][i]==df_test['Y_test'][i]):
                accuracy += 1
                if(df_test['Y'][i]==0):
                    confusion_martix[0][0] += 1
                else:
                    confusion_martix[1][1] += 1
            
            if(df_test['Y'][i] == 0 and df_test['Y_test'][i] == 4):
                confusion_martix[1][0] += 1
            elif(df_test['Y'][i] == 4 and df_test['Y_test'][i] == 0):
                confusion_martix[0][1] += 1
            total += 1
    return (accuracy/total), confusion_martix

In [ ]:
phis, thetas_given_positive, thetas_given_negative = train(df, df1, pos_voc, neg_voc)
# train_accuracy, confusion_matrix_train = test(phis, thetas_given_positive, thetas_given_negative, train_file)
test_accuracy, confusion_matrix_test = test(phis, thetas_given_positive, thetas_given_negative, test_file)
# print('Training Accuracy is ' + str(train_accuracy*100) + '%')
print('Testing Accuracy is ' + str(test_accuracy*100) + '%')
print('Confusion Matrix for Testing Data: \n', confusion_matrix_test)
print('-ve tweets accuracy: %s' % str(confusion_matrix_test[0][0]/(confusion_matrix_test[0][0]+confusion_matrix_test[1][0])))
print('+ve tweets accuracy: %s' % str(confusion_matrix_test[1][1]/(confusion_matrix_test[1][1]+confusion_matrix_test[0][1])))